In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opti
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import pickle
import datetime
import os

data = pd.read_csv("Crop_recommendation.csv")

features = data.iloc[:,:-1].values
input_size = len(features[0])
labels = data.iloc[:,-1].values

encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
num_classes = len(np.unique(labels)) 

features = torch.tensor(features, dtype=torch.float32)
labels = torch.tensor(labels ,dtype=torch.long)

mean = features.mean(dim=0)
std = features.std(dim = 0)
features = (features - mean)/std
np.savez("normalization.npz", mean=mean, std=std)

with open("pickle/encoder.pkl","wb") as file:
    pickle.dump(encoder,file)

class CustomDataset(Dataset):
    def __init__(self,features,labels):
        self.features = features
        self.labels = labels
    def __len__(self):
        return len(self.features)
    def __getitem__(self,index):
        feature = self.features[index]
        label = self.labels[index]
        return feature , label
        
dataset = CustomDataset(features,labels)
train_size = int(0.8*len(dataset))
val_size = len(dataset) - train_size
train_dataset,val_dataset = torch.utils.data.random_split(dataset,[train_size , val_size])

class Neural_network(nn.Module):
    def __init__(self,input_size,num_classes):
        super(Neural_network,self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, num_classes)
    def forward(self, x):
        x = F.selu(self.fc1(x))
        x = F.selu(self.fc2(x))
        x = F.selu(self.fc3(x))
        x = self.fc4(x)
        return F.softmax(x)

network = Neural_network(7,22)
loss_function = nn.CrossEntropyLoss()
optimizer = opti.Adam(network.parameters(),lr= 0.0001)

train_losses = []
val_losses = []
EPOCH = 100
train_accuracies = []
val_accuracies = []

for epoch in range(EPOCH):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_dataset):
        optimizer.zero_grad()
        outputs = network(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_dataset)
    train_losses.append(train_loss)
    with torch.no_grad():
        val_loss = 0.0
        for inputs, labels in val_dataset:
            outputs = network(inputs)
            loss = loss_function(outputs, labels)
            val_loss += loss.item()
        val_loss /= len(val_dataset)
        val_losses.append(val_loss)
    if epoch % 10 == 9:
        print('Epoch :',epoch+1) 
print('Finished training')

C:\Users\pavan\AppData\Local\Temp\ipykernel_40028\1304030784.py:63: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Epoch : 10
Epoch : 20
Epoch : 30
Epoch : 40
Epoch : 50
Epoch : 60
Epoch : 70
Epoch : 80
Epoch : 90
Epoch : 100
Finished training


In [ ]:
from threading import Thread
from flask import Flask, request, render_template
import torch


model_name = "crop_prediction_model"+".hdf5"
torch.save(network.state_dict(),model_name)
model = Neural_network(7,22)

normalization_data = np.load("normalization.npz")
mean = torch.tensor(normalization_data["mean"])
std = torch.tensor(normalization_data["std"])

with open("pickle/encoder.pkl", "rb") as file:
    encoder = pickle.load(file)

model = Neural_network(input_size, num_classes)
model.load_state_dict(torch.load('crop_prediction_model.hdf5'))
model.eval()
app = Flask(__name__)


@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Retrieve input values from the form
        nitrogen = float(request.form['nitrogen'])
        phosphorus = float(request.form['phosphorus'])
        potassium = float(request.form['potassium'])
        temperature = float(request.form['temperature'])
        humidity = float(request.form['humidity'])
        ph = float(request.form['ph'])
        rainfall = float(request.form['rainfall'])

        # Prepare input tensor
        in_features = torch.tensor([nitrogen, phosphorus, potassium, temperature, humidity, ph, rainfall])
        in_features = (in_features - mean) / std

        # Predict crop
        with torch.no_grad():
            output = model(in_features)
            prediction = output.argmax().item()
        
        crop = encoder.inverse_transform([prediction])[0]

        # Return the prediction result
        return f"The recommended crop is: {crop}"
    except Exception as e:
        return f"Error: {str(e)}"

# Add this snippet at the end
def run_app():
    app.run(debug=False, use_reloader=False)

flask_thread = Thread(target=run_app)
flask_thread.start()

C:\Users\pavan\AppData\Local\Temp\ipykernel_40028\4239709321.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('crop_prediction_model.hdf

 * Serving Flask app '__main__'


 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
